<a href="https://colab.research.google.com/github/joony0512/Deep_Learning_Class/blob/main/Part4/P4_Ch04_CH04_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FashionMNIST data 불러오기

In [1]:
import torch
print(torch.__version__)

1.13.1+cu116


In [2]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets 
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np


In [3]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as tr
from torchvision import datasets 


In [4]:
training_data = datasets.FashionMNIST(
    root ='data',
    train = True,
    download =True,
    transform =ToTensor()
)
test_data = datasets.FashionMNIST(
    root ='data',
    train =False,
    download =True,
    transform =ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



# DataLoader 만들기

In [5]:
# DataLoader 만들기
train_dataloader = DataLoader(training_data, batch_size = 64, shuffle =True)
test_dataloader = DataLoader(test_data, batch_size = 64, shuffle = False)

# Model

In [6]:
# device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


## Model class 만들기


In [7]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 128), #tensorflow의 Dense
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(128, 10)
    )

  def forward(self, x): #tensorflow의 call
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [8]:
# Model instance 생성, device 설정
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)


# Trainig /Validation

## Loss Function

In [9]:
# 손실함수 초기화
loss_fn = nn.CrossEntropyLoss()

## Optimizer
update시켜줌

In [10]:
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr =learning_rate)

## Training / Validation(Test) Function

In [11]:
# Training을 위한 함수

def train_loop(dataloader, model, loss_fn, optimizer):  #dataloader를 가져와서 데이터꺼내고, 배치단위로 돌리게함
  size = len(dataloader.dataset) #len을 사용해 데이터 개수를 센다 .60000개
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device) #X와 y를 gpu로 보낸다.
    pred = model(X)                 # 1.모델에 넣기 #model은 위에서 gpu로 보내둠
    loss = loss_fn(pred, y)         # 2.loss계산

    #역전파
    optimizer.zero_grad()           # 3.optimizer에 남아있는 gradient를 clean-up해준다
    loss.backward()                 # 4.backpropagation이 일어남
    optimizer.step()                # 5.step한번마다 weight와 bias update가 일어난다.

    if batch % 100 ==0:
      #100번 돌때마다 화면에 출력
      loss, current = loss.item(), batch*len(X)  #item() : 0차원인 경우 숫자뽑아서 바로 반환
      print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [12]:
def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader) # batch수 계산. 만약 1000개데이터를 100개씩 돌리면 10개의 배치임.
  test_loss, correct = 0,0

  with torch.no_grad(): #업데이트 안해도 되는것들-> 용량 안잡아먹게처리
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss +=loss_fn(pred, y).item()#item() : 0차원인 경우 숫자뽑아서 바로 반환
      correct +=(pred.argmax(1)==y).type(torch.float).sum().item()

  test_loss/=num_batches
  correct/=size
  print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n')

In [14]:
# 학습 진행하기
epochs = 10
for t in range(epochs):
  print(f'Epoch {t+1}\n-------------------------')
  train_loop(train_dataloader, model, loss_fn, optimizer) 
  test_loop(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
-------------------------
loss: 0.474370 [    0/60000]
loss: 0.369840 [ 6400/60000]
loss: 0.468353 [12800/60000]
loss: 0.452639 [19200/60000]
loss: 0.281761 [25600/60000]
loss: 0.402768 [32000/60000]
loss: 0.429661 [38400/60000]
loss: 0.309582 [44800/60000]
loss: 0.553441 [51200/60000]
loss: 0.390886 [57600/60000]
Test Error: 
 Accuracy: 85.0%, Avg loss:0.415425

Epoch 2
-------------------------
loss: 0.372424 [    0/60000]
loss: 0.419230 [ 6400/60000]
loss: 0.553011 [12800/60000]
loss: 0.233859 [19200/60000]
loss: 0.388979 [25600/60000]
loss: 0.544753 [32000/60000]
loss: 0.237189 [38400/60000]
loss: 0.479230 [44800/60000]
loss: 0.242756 [51200/60000]
loss: 0.377402 [57600/60000]
Test Error: 
 Accuracy: 85.7%, Avg loss:0.411234

Epoch 3
-------------------------
loss: 0.399636 [    0/60000]
loss: 0.537516 [ 6400/60000]
loss: 0.199436 [12800/60000]
loss: 0.285716 [19200/60000]
loss: 0.451969 [25600/60000]
loss: 0.582991 [32000/60000]
loss: 0.395264 [38400/60000]
loss: 0.362893 

# Model 저장하고 불러오기

## parameter만 저장하고 불러오기

In [15]:
# 학습된 model parameter 저장
torch.save(model.state_dict(), 'model_weights.pth')

In [20]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [16]:
# 새 Model instance생성, device 설정
model2 = NeuralNetwork().to(device)
print(model2)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [17]:
# test
model2.eval() #model이 트레이닝중이 아니라고 알리는것
test_loop(test_dataloader, model2, loss_fn)

Test Error: 
 Accuracy: 17.1%, Avg loss:2.296867



In [19]:
# 저장한 parameter불러오기
model2.load_state_dict(torch.load('model_weights.pth'))

<All keys matched successfully>

In [21]:
# test
model2.eval()
test_loop(test_dataloader, model2, loss_fn)

Test Error: 
 Accuracy: 88.2%, Avg loss:0.332876



## Model 전체를 저장하고 불러오기

In [22]:
#저장하기
torch.save(model, 'model.pth')

In [23]:
model3 = torch.load('model.pth')
test_loop(test_dataloader, model2, loss_fn)

Test Error: 
 Accuracy: 88.2%, Avg loss:0.332876

